In [2]:
from __future__ import print_function

import json

from os import listdir
from os.path import isfile, join
import itertools

from datetime import datetime, timedelta
import re

import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='vinicezarml', api_key='nanEOHpctjmzWH6sokGZ')
plotly.tools.set_config_file(world_readable=True, sharing='public')


In [3]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [4]:
# https://www.iana.org/assignments/language-subtag-registry/language-subtag-registry

dic_languages = {'ar': 'Arabic',
 'ca': 'Catalan',
 'da': 'Danish',
 'de': 'German',
 'el': 'Greek',
 'et': 'Estonian',
 'en': 'English',
 'en-gb': 'English UK',
 'es': 'Spanish',
 'fa': 'Farsi',
 'fi': 'Finnish',
 'fil': 'Filipino',
 'fr': 'French',
 'he': 'Hebrew',
 'hi': 'Hindi',
 'hu': 'Hungarian',
 'id': 'Indonesian',
 'it': 'Italian',
 'ja': 'Japanese',
 'ko': 'Korean',
 'msa': 'Malay',
 'nl': 'Dutch',
 'no': 'Norwegian',
 'pl': 'Polish',
 'pt': 'Portuguese',
 'ru': 'Russian',
 'sv': 'Swedish',
 'th': 'Thai',
 'tr': 'Turkish',
 'tl': 'Filippino',                
 'ur': 'Urdu',
 'zh-cn': 'Simplified Chinese',
 'zh-tw': 'Traditional Chinese'}
    

### Loading Data

In [5]:
def centroid(vertexes):
    _x_list = [vertex [0] for vertex in vertexes]
    _y_list = [vertex [1] for vertex in vertexes]
    _len = len(vertexes)
    _x = sum(_x_list) / _len
    _y = sum(_y_list) / _len
    return(_x, _y)

point = centroid([[25.195932, 36.220559], [25.195932, 36.631589], [25.69548, 36.631589], [25.69548, 36.220559]])
print(centroid([[25.195932, 36.220559], [25.195932, 36.631589], [25.69548, 36.631589], [25.69548, 36.220559]]))

(25.445706, 36.426074)


In [6]:
# equivalent to centroid fuction above
from shapely.geometry import Polygon

p1 = Polygon([[25.195932, 36.220559], [25.195932, 36.631589], [25.69548, 36.631589], [25.69548, 36.220559]])

print(p1.centroid)
print(type(p1.centroid))

POINT (25.445706 36.42607400000001)
<class 'shapely.geometry.point.Point'>


In [7]:
def isSantorini(point):
    lon = point[0]
    lat = point[1]
    if (lon>25.27 and lon<25.52) and (lat>36.30 and lat<36.50):
        return True
    else:
        return False
    
    
isSantorini(point)

True

In [14]:
# directory from the streaming api
directory = '/home/master/santorini-tweets/'

# get all files of the directory
files = sorted([f for f in listdir(directory) if isfile(join(directory, f))])

# all tweets
tweets = []

# current idx
i = 0

# load each file in the directory
for file_tweet in files:
    with open(directory + file_tweet, 'r') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                flat_tweet = dict()
                flat_tweet['id'] = tweet['id']
                
                if 'place' in tweet and tweet['place'] is not None: 
                    flat_tweet['place_full_name'] = tweet['place']['full_name'] 
                    flat_tweet['place_name'] = tweet['place']['name'] 
                    flat_tweet['place_country'] = tweet['place']['country'] 
                    flat_tweet['place_country_code'] = tweet['place']['country_code'] 
                    flat_tweet['place_place_type'] = tweet['place']['place_type']
                    flat_tweet['place_bounding_box'] = tweet['place']['bounding_box']['coordinates'][0] 
                    flat_tweet['place_bounding_box_centroid'] = centroid(tweet['place']['bounding_box']['coordinates'][0])
                    flat_tweet['is_bounding_box_in_santorini'] = isSantorini(flat_tweet['place_bounding_box_centroid'])
                else:
                    flat_tweet['place_full_name'] = None
                    flat_tweet['place_name'] = None
                    flat_tweet['place_country'] = None
                    flat_tweet['place_country_code'] = None
                    flat_tweet['place_bounding_box'] = None
                    flat_tweet['place_place_type'] = None
                    flat_tweet['place_bounding_box_centroid'] = None
                    flat_tweet['is_bounding_box_in_santorini'] = None
                
                
                flat_tweet['lang'] = tweet['lang']
                flat_tweet['text'] = tweet['text']
                
                if 'entities' in tweet and 'media' in tweet['entities']:
                    flat_tweet['media_url'] = tweet['entities']['media'][0]['media_url'] if tweet['entities']['media'] is not None else None
                    flat_tweet['media_type'] = tweet['entities']['media'][0]['type'] if tweet['entities']['media'] is not None else None
                
                flat_tweet['geo_coordinates'] = tweet['geo']['coordinates'] if 'geo' in tweet and tweet['geo'] is not None else None
                if flat_tweet['geo_coordinates'] is not None:
                    swap_coords = [flat_tweet['geo_coordinates'][1], flat_tweet['geo_coordinates'][0]]
                    flat_tweet['is_geo_coord_in_santorini'] = isSantorini(swap_coords)
                else:
                    flat_tweet['is_geo_coord_in_santorini'] = None
                
                flat_tweet['user_location'] = tweet['user']['location']
                flat_tweet['user_screen_name'] = tweet['user']['screen_name']
                flat_tweet['user_id'] = tweet['user']['id']
                
                flat_tweet['created_at'] = tweet['created_at']
                tweets.append(flat_tweet)
                del tweet
            except Exception as e: 
                print("Unexpected error:", e)

    i = i + 1   
    update_progress(i / len(files))


Progress: [##################################################] 100.0%


In [9]:
for i,t in enumerate(tweets):
    print(t)
    if i>10: break

{'place_country': 'Greece', 'place_bounding_box_centroid': (24.508861, 38.2752785), 'geo_coordinates': None, 'place_bounding_box': [[19.372958, 34.802021], [19.372958, 41.748536], [29.644764, 41.748536], [29.644764, 34.802021]], 'lang': 'en', 'id': 993830134963634176, 'place_place_type': 'country', 'user_location': 'Florina, Greece', 'is_geo_coord_in_santorini': None, 'created_at': 'Tue May 08 12:29:03 +0000 2018', 'place_country_code': 'GR', 'user_id': 871024533318361089, 'place_name': 'Greece', 'text': 'This three in one plant hanger is a perfect addition to your balcony or patio! You can even hang it indoors! Check… https://t.co/0TAUJ8SQmW', 'is_bounding_box_in_santorini': False, 'place_full_name': 'Greece', 'user_screen_name': 'SanquilArt'}
{'place_country': None, 'place_bounding_box_centroid': None, 'geo_coordinates': None, 'place_bounding_box': None, 'lang': 'en', 'id': 993830238051237888, 'place_place_type': None, 'media_type': 'photo', 'media_url': 'http://pbs.twimg.com/media/D

In [22]:
# computing the geo,place,non-tagged  
len_tweets_geotagged = len([x for x in tweets if 'geo_coordinates' in x and x['geo_coordinates'] is not None])
len_tweets_placetagged = len([x for x in tweets if 'place_bounding_box' in x and x['place_bounding_box'] is not None])
len_tweets_non_geotagged = len([x for x in tweets if 'geo_coordinates' in x and x['geo_coordinates'] is None])
len_tweets_santorini = len([x for x in tweets if ('is_bounding_box_in_santorini' in x and x['is_bounding_box_in_santorini'] is True) or ('is_geo_coord_in_santorini' in x and x['is_geo_coord_in_santorini'] is True)])

print("Geo-tagged tweets", len_tweets_geotagged)
print("Tweets with place", len_tweets_placetagged)
print("Non-geo-tagged tweets", len_tweets_non_geotagged)
print("Tweets in Santorini", len_tweets_santorini)

Geo-tagged tweets 45936
Tweets with place 555703
Non-geo-tagged tweets 4860920
Tweets in Santorini 55006


In [29]:
len_tweets_geotagged = len([x for x in tweets if 'geo_coordinates' in x and x['geo_coordinates'] is not None])
len_tweets_geotagged_in_Santorini = len([x for x in tweets if ('geo_coordinates' in x and x['geo_coordinates'] is not None) and  ('is_geo_coord_in_santorini' in x and x['is_geo_coord_in_santorini'] is True)])
len_tweets_place_in_Santorini = len([x for x in tweets if ('is_bounding_box_in_santorini' in x and x['is_bounding_box_in_santorini'] is True)])

In [30]:
print("Geo-tagged tweets", len_tweets_geotagged)
print("Tweets with geo", len_tweets_geotagged_in_Santorini)
print("Tweets with place", len_tweets_place_in_Santorini)

Geo-tagged tweets 45936
Tweets with geo 18046
Tweets with place 54449


In [18]:
len(tweets)

4906856

In [20]:
with open('../../datasets/santorini_tweet_ids.txt', 'w') as f:
    for x in tweets:
        if ('is_bounding_box_in_santorini' in x and x['is_bounding_box_in_santorini'] is True) or ('is_geo_coord_in_santorini' in x and x['is_geo_coord_in_santorini'] is True):
            f.write(str(x['id']) + "\n")

In [21]:
with open('../../datasets/santorini_geo_tweet_ids.txt', 'w') as f:
    for x in tweets:
        if ('is_geo_coord_in_santorini' in x and x['is_geo_coord_in_santorini'] is True):
            f.write(str(x['id']) + "\n")

### Older Stats:

Geo-tagged tweets 45046
Tweets with place 546258
Non-geo-tagged tweets 4793980
Tweets in Santorini 53166

### Saving the tweets_geotagged

In [23]:
def isValid(tweet):
    if ('geo_coordinates' in tweet and tweet['geo_coordinates'] is not None) \
        or ('place_bounding_box' in tweet and tweet['place_bounding_box'] is not None):
        return True
    else:
        return False
    
print(tweets[0])    
isValid(tweets[0])

{'place_country': 'Greece', 'place_bounding_box_centroid': (24.508861, 38.2752785), 'geo_coordinates': None, 'place_bounding_box': [[19.372958, 34.802021], [19.372958, 41.748536], [29.644764, 41.748536], [29.644764, 34.802021]], 'lang': 'en', 'id': 993830134963634176, 'place_place_type': 'country', 'user_location': 'Florina, Greece', 'is_geo_coord_in_santorini': None, 'created_at': 'Tue May 08 12:29:03 +0000 2018', 'place_country_code': 'GR', 'user_id': 871024533318361089, 'place_name': 'Greece', 'text': 'This three in one plant hanger is a perfect addition to your balcony or patio! You can even hang it indoors! Check… https://t.co/0TAUJ8SQmW', 'is_bounding_box_in_santorini': False, 'place_full_name': 'Greece', 'user_screen_name': 'SanquilArt'}


True

In [25]:
import pickle

tweets_geo_tagged_place = [x for x in tweets if isValid(x)]
with open('../../datasets/tweets_geotagged_place.pickle', 'wb') as file:
    pickle.dump(tweets_geo_tagged_place, file)

print(len(tweets_geo_tagged_place))
del tweets_geo_tagged_place

556580
